In [21]:
import numpy as np

import mne

print(__doc__)

In [25]:
sfreq = 125  # Sampling frequency
times = np.arange(0, 1, (1/sfreq)) 

sin = np.sin(times * 10)  # Multiplied by 10 for shorter cycles
cos = np.cos(times * 10)
sinX2 = sin * 2
cosX2 = cos * 2

# Numpy array of size 4 X 10000.
data = np.array([sin, cos, sinX2, cosX2])

# Definition of channel types and names.
ch_types = ['mag', 'mag', 'grad', 'grad']
ch_names = ['sin', 'cos', 'sinX2', 'cosX2']

In [3]:
# sfreq = 1000  # Sampling frequency
times = np.arange(0, 10, 0.001)  # Use 10000 samples (10s)

sin = np.sin(times * 10)  # Multiplied by 10 for shorter cycles
cos = np.cos(times * 10)
sinX2 = sin * 2
cosX2 = cos * 2

# Numpy array of size 4 X 10000.
data = np.array([sin, cos, sinX2, cosX2])

# Definition of channel types and names.
ch_types = ['mag', 'mag', 'grad', 'grad']
ch_names = ['sin', 'cos', 'sinX2', 'cosX2']

In [29]:
data.shape

In [30]:
# It is also possible to use info from another raw object.
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

In [31]:
info

In [32]:
raw = mne.io.RawArray(data, info)

# Scaling of the figure.
# For actual EEG/MEG data different scaling factors should be used.
scalings = {'mag': 2, 'grad': 2}

raw.plot(n_channels=4, scalings=scalings, title='Data from arrays',
         show=True, block=True)

# It is also possible to auto-compute scalings
scalings = 'auto'  # Could also pass a dictionary with some value == 'auto'
raw.plot(n_channels=4, scalings=scalings, title='Auto-scaled Data from arrays',
         show=True, block=True)

In [2]:
# Author: Teon Brooks <teon.brooks@gmail.com>
#
# License: BSD (3-clause)
import matplotlib.pyplot as plt

from mne.datasets import sample
from mne.io import read_raw_fif

from mne_realtime import LSLClient, MockLSLStream

print(__doc__)

# this is the host id that identifies your stream on LSL
host = 'UN-2023.04.61'
# this is the max wait time in seconds until client connection
wait_max = 5


# Load a file to stream raw data
data_path = sample.data_path()
raw_fname = data_path  / 'MEG' / 'sample' / 'sample_audvis_filt-0-40_raw.fif'
raw = read_raw_fif(raw_fname).crop(0, 30).load_data().pick('eeg')

# For this example, let's use the mock LSL stream.
_, ax = plt.subplots(1)
n_epochs = 5

# main function is necessary here to enable script as own program
# in such way a child process can be started (primarily for Windows)
if __name__ == '__main__':
    with MockLSLStream(host, raw, 'eeg'):
        with LSLClient(info=raw.info, host=host, wait_max=wait_max) as client:
            client_info = client.get_measurement_info()
            sfreq = int(client_info['sfreq'])

            # let's observe ten seconds of data
            for ii in range(n_epochs):
                print('Got epoch %d/%d' % (ii + 1, n_epochs))
                plt.cla()
                epoch = client.get_data_as_epoch(n_samples=sfreq)
                epoch.average().plot(axes=ax)
                plt.pause(1.)
            plt.draw()
print('Streams closed')

In [2]:
%matplotlib qt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
# sample DataFrame with EEG data (replace this with your actual DataFrame)
# Assuming columns are 'timestamp', 'channel1', 'channel2', ..., 'channel8'
# and rows are successive time points
data = pd.DataFrame({
    'timestamp': pd.date_range('2024-05-23', periods=100, freq='1s'),
    'channel1': np.random.rand(100),
    'channel2': np.random.rand(100),
    'channel3': np.random.rand(100),
    'channel4': np.random.rand(100),
    'channel5': np.random.rand(100),
    'channel6': np.random.rand(100),
    'channel7': np.random.rand(100),
    'channel8': np.random.rand(100)
})

# Set up plot
fig, ax = plt.subplots(2, 4, figsize=(12, 6))

# Function to update plot
def update_plot(frame):
    ax.clear()
    for i in range(8):
        ax[i//4, i%4].plot(data['timestamp'][:frame], data[f'channel{i+1}'][:frame])
        ax[i//4, i%4].set_title(f'Channel {i+1}')
        ax[i//4, i%4].set_xlabel('Time')
        ax[i//4, i%4].set_ylabel('Signal')
        ax[i//4, i%4].grid(True)

# Update plot in real-time
ani = FuncAnimation(fig, update_plot, frames=len(data), interval=1000) # Update every 1 second (adjust as needed)
plt.tight_layout()
plt.show()